## Operations Reserach Club
## Optimization with Google OR tools

Please visit the following website for the tutorials and documentation. 

https://developers.google.com/optimization/introduction/python

We will maximize the usage of the operations room of a hospital. It has multiple op-rooms and multiple departments that are doing the operations in those op-rooms.

In [4]:
from ortools.linear_solver import pywraplp

In [5]:
def create_data_model():
    """Create the data for the example."""
    data = {}
    weights1 = [48, 30, 42]
    weights2 = [36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36, 36]
    values1 =  [10, 30, 25]
    values2 =  [50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 30, 25]
    data['weights1'] = weights1
    data['values1'] = values1
    data['items1'] = list(range(len(weights1)))
    data['num_items1'] = len(weights1)
    
    data['weights2'] = weights2
    data['values2'] = values2
    data['items2'] = list(range(len(weights2)))
    data['num_items2'] = len(weights2)   
    
    num_bins = 5
    data['bins'] = list(range(num_bins))
    data['bin_capacities'] = [100, 100, 100, 100, 100]
    return data

In [6]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [7]:
data=create_data_model()
data

{'weights': [48, 30, 42, 36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36, 36],
 'values': [10, 30, 25, 50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 30, 25],
 'items': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'num_items': 15,
 'bins': [0, 1, 2, 3, 4],
 'bin_capacities': [100, 100, 100, 100, 100]}

In [8]:
# Variables
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i in data['items']:
    for j in data['bins']:
        x[(i, j)] = solver.IntVar(0, 1, 'x_%i_%i' % (i, j))

x

In [10]:
# Constraints
# Each item can be in at most one bin.
for i in data['items']:
    solver.Add(sum(x[i, j] for j in data['bins']) <= 1)
# The amount packed in each bin cannot exceed its capacity.
for j in data['bins']:
    solver.Add(
        sum(x[(i, j)] * data['weights'][i]
            for i in data['items']) <= data['bin_capacities'][j])

In [11]:
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 20


In [15]:
# Objective
objective = solver.Objective()

for i in data['items']:
    for j in data['bins']:
        objective.SetCoefficient(x[(i, j)], data['values'][i])
objective.SetMaximization()

In [16]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Total packed value:', objective.Value())
    total_weight = 0
    for j in data['bins']:
        bin_weight = 0
        bin_value = 0
        print('Bin ', j, '\n')
        for i in data['items']:
            if x[i, j].solution_value() > 0:
                print('Item', i, '- weight:', data['weights'][i], ' value:',
                      data['values'][i])
                bin_weight += data['weights'][i]
                bin_value += data['values'][i]
        print('Packed bin weight:', bin_weight)
        print('Packed bin value:', bin_value)
        print()
        total_weight += bin_weight
    print('Total packed weight:', total_weight)
else:
    print('The problem does not have an optimal solution.')

Total packed value: 395.00000000000006
Bin  0 

Item 1 - weight: 30  value: 30
Item 10 - weight: 30  value: 45
Item 13 - weight: 36  value: 30
Packed bin weight: 96
Packed bin value: 105

Bin  1 

Item 7 - weight: 42  value: 40
Item 12 - weight: 42  value: 20
Packed bin weight: 84
Packed bin value: 60

Bin  2 

Item 2 - weight: 42  value: 25
Item 5 - weight: 48  value: 30
Packed bin weight: 90
Packed bin value: 55

Bin  3 

Item 3 - weight: 36  value: 50
Item 8 - weight: 36  value: 30
Item 9 - weight: 24  value: 35
Packed bin weight: 96
Packed bin value: 115

Bin  4 

Item 4 - weight: 36  value: 35
Item 14 - weight: 36  value: 25
Packed bin weight: 72
Packed bin value: 60

Total packed weight: 438
